In [ ]:
import datetime as dt
from pathlib import Path

import matplotlib as mpl
import matplotlib.figure
import numpy as np
import pandas as pd

%matplotlib inline

## A few short snippets used in Voyager data processing

In [ ]:
def read_rate_file(ftpath: Path, rename: bool = True, uncertainties: bool = False) -> pd.DataFrame:
    """
    Read a gzipped rates file from ftecs and create a DataFrame with a DatetimeIndex
    from the start time in columns 0-4.  Clean up the ftecs column names if rename
    is True and include the U uncertainties columns if uncertainties is True.

    :param ftpath: Path to the ftecs rates file
    :param rename: Do not include the S0n sector in the colum names if True
    :param uncertainties: Include the U columns if True
    :return: DataFrame ceated from the rates data with a DatetimeIndex
    """
    if not ftpath.exists():
        print(f'{ftpath} not found')
        return pd.DataFrame()
    df = pd.read_csv(
        ftpath,
        compression='gzip',
        index_col=[0, 1, 2, 3, 4],
        na_values=-1,
    )
    if df.empty:
        return df
    df.index = pd.to_datetime(
        ['{} {} {} {} {}'.format(*tup) for tup in df.index.to_numpy()],
        format='%Y %j %H %M %S.%f',
    )
    df. insert(5, 'End_date', pd.to_datetime(
        ['{} {} {} {} {}'.format(*tup) for tup in df.iloc[:, :5].to_numpy()],
        format='%Y.0 %j.0 %H.0 %M.0 %S.%f',
    ))
    ratename = df.iloc[0, [7]].index[0]
    ftrate = ratename[: ratename.find('RS0')]
    if rename:
        df.columns = df.columns.str.replace(f'{ftrate}R', ftrate[-2:])
    if not uncertainties:
        df = df[[col for col in df.columns if not col[-1] == 'U']]
    return df.loc[:, 'End_date' : df.columns[-3]]

####
ftfile = Path('data/V11999001B68.CSV.GZ')
ratesdf = read_rate_file(ftfile)
ratesdf.head()

##### S0n in the rate column names is the motor step.  This routine "pivots" the DataFrame to put the rate data and motor step into columns.  Useful for combining rate data from multiple Brr files into a format that's easier to work with

In [ ]:
def step_to_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a DataFrame containing a Step column containing the motor step and
    a column containing the rate data.  There is probably a better way to do this
    :param df: DataFrame containing rates data starting in column 2 labeled
               RateS0n where n is the motor step
    :return: DataFrame with Rate and motor step in columns 2 and 3
    """
    rate = df.columns[3]
    rate = rate[: rate.find('S0')]
    step_df = (
        df.iloc[:, :2]
          .join(df.iloc[:, 2:]
                  .stack()
                  .reset_index(level=1)
                )
          .rename(columns={'level_1': 'Step', 0: rate})
    )
    step_df['dd'] = step_df['dd'].fillna(-1)
    step_df['Step'] = step_df['Step'].str[-1].astype(int)
    return step_df


####
# read singles rates files, convert to Rate & Step format and concat to make
# a single DataFrame with step and rates data
(pd.concat(
     [
         step_to_column(read_rate_file(Path(f'data/V11999001B{rr}.CSV.GZ')))
         for rr in range(68, 76)
     ],
 )
   .rename_axis('SCET')
   .sort_values(by=['SCET', 'dd', 'Step'])
   .head(13)
)


#### The Voyager pipeline updates text files containing flux data and plots of the data.  These snippets read the data from the flux file and plots it

In [ ]:
def readdb(dbfile: Path):
    with open(dbfile, 'r') as dbfile:
        # read 7 header lines and keep the 8th line containing column names
        for _ in range(8):
            line = dbfile.readline()
        colnames = line[1:].split()
        usecols = ['s/c', 'Start_Year', 'Start_Day', 'End_Year', 'End_Day',
                   'Std._Year', 'Z', 'E/n', 'Emin', 'Emax', '(delta)E/2',
                   'Flux', 'sigma']
        df = (pd.read_csv(dbfile,
                          skiprows=1,
                          delim_whitespace=True,
                          usecols=usecols,
                          names=colnames,
                          index_col=['s/c', 'Start_Year', 'Start_Day', 'Z'],
                          )
                .rename(columns={'(delta)E/2': '∆E/2'})
                .sort_index()
              )
    return df

####
readdb(dbfile=Path('data/vmaster_year.txt')).head()

In [ ]:
traces = {1: ('H', 'o', 7, 'r'),
          2: ('He', 's', 7, 'darkviolet'),
          6: ('C', 'd', 7, 'k'),
          7: ('N', 'D', 6, 'b'),
          8: ('O', '^', 7, 'lime'),
          10: ('Ne', '<', 7, 'peru'),
          12: ('Mg', 'H', 7, 'fuchsia'),
          14: ('Si', '>', 7, 'seagreen'),
          16: ('S', 'p', 8, 'darkorange'),
          18: ('Ar', '*', 11, 'aqua'),
          26: ('Fe', 'v', 7, 'crimson'),
          }

def restore_minor_ticks_log_plot(
        ax=None, n_subticks=9
    ) -> None:
        """For axes with a logrithmic scale where the span (max-min) exceeds
        10 orders of magnitude, matplotlib will not set logarithmic minor ticks.
        If you don't like this, call this function to restore minor ticks.

        Args:
            ax:
            n_subticks: Number of Should be either 4 or 9.

        Returns:
            None
        https://stackoverflow.com/questions/44078409/matplotlib-semi-log-plot-minor-tick-marks-are-gone-when-range-is-large
        """
        if ax is None:
            print('you must pass an ax to restore_minor_ticks_log_plot')
            return
        # Method from SO user importanceofbeingernest at
        # https://stackoverflow.com/a/44079725/5972175
        locmaj = mpl.ticker.LogLocator(base=10, numticks=1000)
        ax.xaxis.set_major_locator(locmaj)
        locmin = mpl.ticker.LogLocator(
            base=10.0, subs=np.linspace(0, 1.0, n_subticks + 2)[1:-1], numticks=1000
        )
        ax.yaxis.set_minor_locator(locmin)
        ax.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())

def plot_a_spectra(df: pd.DataFrame, sc: int, year: int, start_day: int):
    end_day = df.loc[(start_day,), 'End_Day'].unique()[0]
    fig = mpl.figure.Figure(figsize=(7.5, 5.5), )
    ax = fig.add_subplot(1, 1, 1, xscale='log', yscale='log',)
    ax.set_position([0.135, 0.145, 0.68, 0.725])
    for z in df.index.get_level_values('Z').unique():
        (elem, mrkr, siz, colr) = traces.get(z)
        zdf = df.loc[(start_day, z), :]
        ax.errorbar(zdf['E/n'], zdf['Flux'],
                    yerr=zdf['sigma'], xerr=zdf['∆E/2'],
                    label=elem, marker=mrkr, ms=siz, lw=0,
                    mfc='none', mec=colr, mew=0.6,
                    elinewidth=0.75, ecolor=colr,
                    )
    if sc == 1:
        bboxx = 1.003
    else:
        bboxx = 0.92
    # https://stackoverflow.com/questions/14297714/matplotlib-dont-show-errorbars-in-legend
    handles, labels = ax.get_legend_handles_labels()
    handles = [h[0] if isinstance(h, mpl.container.ErrorbarContainer) else h for h in handles]
    ax.legend(handles, labels, fontsize=12, ncol=3 - sc, frameon=False,
              loc='center right',
              bbox_to_anchor=(bboxx, 0.51), bbox_transform=fig.transFigure,
              labelcolor=[val[-1] for val in traces.values()],
              handletextpad=0.01, columnspacing=0.25, labelspacing=.25)
    ax.text(.20, 25, f'Voyager {sc}\nLECP', ha='center',
            fontsize=18, fontweight='medium')
    ax.set_title(f'{year}: {start_day} – {end_day}', y=1.02,
                 fontsize=26, fontweight='medium')
    ax.text(24, 24, 'Exclude Sectors 4,8,U', fontsize=8, c='r')
    ax.set_ylabel('Flux ( 1 / $\\mathdefault{cm^{2}}$–sr–s–MeV / nuc )', fontsize=14)
    ax.set_xlabel('Energy / nucleon (MeV / nuc)', fontsize=14)
    ax.tick_params(axis='both', labelsize=12)
    ax.tick_params(axis='both', which='major', right=True, top=True, length=7.5)
    ax.tick_params(axis='both', which='minor', top=True, left=True, right=True, length=4)

    ax.set_xlim([0.1, 100])
    ax.set_ylim([1e-8, 10])
    ax.set_yticks(np.logspace(-8, 1, 10))
    restore_minor_ticks_log_plot(ax=ax)
    tick_labels = [f'$\mathdefault{{10^{{{decade}}}}}$' for decade in range(-8, 2)]
    ax.yaxis.set_ticklabels(tick_labels, ha='left')
    ax.yaxis.set_tick_params(pad=27)
    scalar_fmtr = mpl.ticker.ScalarFormatter()
    ax.xaxis.set_major_formatter(scalar_fmtr)

    ax.text(0.036, 1.3e-9, f'{dt.datetime.today():%b %d, %Y}', fontsize=7)
    return fig

####
sc = 1
year = 1999
specdf = readdb(dbfile=Path('data/vmaster_year.txt')).loc[sc, year]
plot_a_spectra(df=specdf, sc=sc, year=year, start_day=1)
